### Notebook that shows how to make an API call to return predictions from trained model

In [1]:
# Libraries:
import pandas as pd
import requests

### 1. Example of a GET request

In [2]:
# Define application URL:
url = "http://10.99.191.249:8080"

# Define the get URL:
endpoint_name = '/metadata/fecha_actual'
get_url = url + endpoint_name

# Call API and read response:
response = requests.get(get_url)
result = response.json()

print(result)

{'fecha_actual': '2023-02-08 11:39:39'}


### 2. Example of a POST request

In [4]:
# Define the post URL:
endpoint_name = '/predict'
post_url = url + endpoint_name

# Call API and read response:
response = requests.post(post_url, json = {"colA":4.5, "colB":66, "colC":9.98, "colD":3.34})
result = response.json()

if response.status_code == 200:
    print('Llamada correcta:', response.status_code)
    print(result)
else:
    print("URL is not responding", response.status_code)

Llamada correcta: 200
{'pred_class': 2.0, 'pred_prob': 0.89}


### 3. Full example

In [13]:
# Create a sample Pandas DataFrame of new rows and predict them:

df_to_predict = pd.DataFrame(data = [[1, 3.2, 44, 10],
                                     [0.2, 4, 0.6, 6.66],
                                     [8, 1.111, 7.55, 3]],
                             
                             columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
                             )

df_to_predict

,sepal_length,sepal_width,petal_length,petal_width
0,1.0,3.200,44.00,10.00
1,0.2,4.000,0.60,6.66
2,8.0,1.111,7.55,3.00


In [14]:
# Define new columns to store predictions and prediction date:

df_to_predict['predicted_class'] = None
df_to_predict['predicted_probability'] = None
df_to_predict['predict_date'] = None

df_to_predict

,sepal_length,sepal_width,petal_length,petal_width,predicted_class,predicted_probability,predict_date
0,1.0,3.200,44.00,10.00,None,None,None
1,0.2,4.000,0.60,6.66,None,None,None
2,8.0,1.111,7.55,3.00,None,None,None


In [15]:
# Call API for each row and save the response

for i, row in df_to_predict.iterrows():
    
    # Call API:
    response_get = requests.get(get_url)
    result_get = response_get.json()
    
    response_post = requests.post(post_url, json = df_to_predict.loc[i][:-3].to_dict())
    result_post = response_post.json()
    
    # Insert response into DataFrame:
    df_to_predict.at[i, 'predicted_class'] = result_post['pred_class']
    df_to_predict.at[i, 'predicted_probability'] = result_post['pred_prob']
    df_to_predict.at[i, 'predict_date'] = result_get['fecha_actual']

if response_post.status_code == 200:
    print('Llamada correcta:', response_post.status_code)
else:
    print("URL is not responding:", response_post.status_code)

Llamada correcta: 200


In [16]:
# Show new DF:
df_to_predict

,sepal_length,sepal_width,petal_length,petal_width,predicted_class,predicted_probability,predict_date
0,1.0,3.200,44.00,10.00,2,0.89,2023-02-08 11:41:12
1,0.2,4.000,0.60,6.66,0,0.48,2023-02-08 11:41:12
2,8.0,1.111,7.55,3.00,2,0.99,2023-02-08 11:41:12
